# Introduzione a Python

* Fate sempre riferimento alla [documentazione on-line](https://docs.python.org/3/index.html), 
* L'ambiente didattico è basato su [Jupyter](https://jupyter.org/), chi non vuole installare nulla può iniziare col [playground](https://mybinder.org/v2/gh/let-unimi/playground/master?filepath=Playground.ipynb).

Questo notebook può essere [posto in esecuzione](https://mybinder.org/v2/gh/let-unimi/handouts/master?filepath=L02.ipynb) su [binder](https://mybinder.org/v2/gh/let-unimi/handouts/master?filepath=L02.ipynb).

In [1]:
1+1

2

## Tipi elementari, espressioni, condizionali e cicli

* interi, float (divisione intera), stringhe,
* `if` (statement e ternario) e `while`.

In [2]:
x = 1
while x < 10:
    if x % 2 == 0:
        print(x)
    x += 1

2
4
6
8


## Liste, insiemi e dizionari

* liste (e tuple),
* insiemi,
* dizionari.

In [3]:
d = {
    ('massimo', 'santini'): 105, 
    'carlotta': [2, 7]
}
d[('massimo','santini')]

105

## Iterabili e comprehension

* `for` su iterabili/iteratori (es. `range`),
* `iter` e `next`,
* [comprehension](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).

In [4]:
for elem in {1, 'a', (2,3)}:
    print(elem)

a
1
(2, 3)


In [5]:
for elem in 'ciao mamma mia'.split():
    print(elem)

ciao
mamma
mia


In [6]:
it = iter('mamma mia')

while True:
    elem = next(it, None)
    if elem is ' ': break
    print(elem)
    
print('Ho incontrato uno spazio')    

while True:
    elem = next(it, None)
    if elem is None: break
    print(elem)    

m
a
m
m
a
Ho incontrato uno spazio
m
i
a


In [7]:
quadrati = [x * x for x in range(10)]
quadrati

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [8]:
mezzi = {x // 2 for x in range(10)}
mezzi

{0, 1, 2, 3, 4}

In [9]:
[x * x for x in range(10) if x % 2 == 0]

[0, 4, 16, 36, 64]

In [10]:
for x, y in [(x, y) for x in range(3) for y in range(2) if x >= y]:
    print(x + y)

0
1
2
2
3


In [11]:
a, *b = (1, 2, 3, 4, 5)
print(b)

[2, 3, 4, 5]


## Funzioni

* definizione e invocazione,
* funzioni come tipo di dato,
    * come argomenti, per il [visitor pattern](https://en.wikipedia.org/wiki/Visitor_pattern),
    * per costruire [dispatch table](https://en.wikipedia.org/wiki/Dispatch_table),
    * funzioni *interne* per le [chiusure lessicali](https://en.wikipedia.org/wiki/Closure_(computer_programming)),
* [decoratori](https://docs.python.org/3/reference/compound_stmts.html#function-definitions).

In [12]:
def saluta(chi):
    print('Ciao ', chi)
    
saluta('Massimo')

una_funzione = saluta

una_funzione('Piero')

Ciao  Massimo
Ciao  Piero


In [13]:
dati = [1, 2, 3, 4, 5]

def stampa(dati):
    for dato in dati: print(dato)

def quadra(dati):
    for dato in dati: print(dato*dato)
        
quadra(dati)

1
4
9
16
25


In [14]:
def visita(dati, visitatore):
    for dato in dati:
        visitatore(dato)
        
visita(dati, print)

1
2
3
4
5


In [15]:
def somma(a, b):
    return a + b

def prodotto(a, b):
    return a * b

def sottrazione(a, b):
    return a - b

DISPATCH_TABLE = {
    '+': somma,
    '-': sottrazione
}

it = iter('1 + 2 - 3'.split())

val = int(next(it))
while True:
    op = next(it, None)
    if op is None: break
    right = next(it, None)
    val = DISPATCH_TABLE[op](val, int(right))
print(val)

0


In [16]:
def crea_saluto(saluto):
    def saluta(chi):
        print(saluto, chi)
    return saluta

saluto_informale = crea_saluto('Ciao')
saluto_formale = crea_saluto('Buongiorno')

saluto_formale('prof. Santini')
saluto_informale('Mariuccio')

Buongiorno prof. Santini
Ciao Mariuccio


In [17]:
def mio_range():
    n = 0
    def f():
        nonlocal n
        n += 1
        return n
    return f

prossimo = mio_range()
print(prossimo())
print(prossimo())
print(prossimo())

1
2
3


In [18]:
def rendi_parlante(f):
    def g(x):
        print(f, x)
        val = f(x)
        print('restituisce', val)
        return val
    return g

@rendi_parlante
def raddoppia(x):
    return 3 * x

raddoppia(3)

<function raddoppia at 0x107025c80> 3
restituisce 9


9

In [19]:
chiamate = 0

def conta_chiamate(f):
    def g(x):
        global chiamate
        chiamate += 1
        return f(x)
    return g

@conta_chiamate
def saluta(x):
    print('Ciao', x)
    
saluta('mario')
saluta('piero')
saluta('luca')
print(chiamate)

Ciao mario
Ciao piero
Ciao luca
3


## Oggetti (not really)

In [20]:
'pippo'.upper()

'PIPPO'

# Tecniche di programmazione

## Ricorsione

In [21]:
lol = [2, ['pippo', 4, [5, 6], 7]]

def mostra_chiamate(f):
    depth = -1
    def f_decorata(x):
        nonlocal depth
        depth += 1
        print(' ' * depth, f.__name__, x)
        val = f(x)
        depth -= 1
        return val
    return f_decorata

@mostra_chiamate
def stampa(lol):
    for elem in lol:
        if isinstance(elem, list):
            stampa(elem)
        else:
            print(elem)

stampa(lol)


 stampa [2, ['pippo', 4, [5, 6], 7]]
2
  stampa ['pippo', 4, [5, 6], 7]
pippo
4
   stampa [5, 6]
5
6
7


In [22]:
@mostra_chiamate
def fatt(n):
    if n == 0: return 1
    return n * fatt(n-1)

fatt(5)

 fatt 5
  fatt 4
   fatt 3
    fatt 2
     fatt 1
      fatt 0


120

In [23]:
@mostra_chiamate
def fib(n):
    if n == 0 or n == 1: return 1
    return fib(n-1) + fib(n-2)

fib(5)

 fib 5
  fib 4
   fib 3
    fib 2
     fib 1
     fib 0
    fib 1
   fib 2
    fib 1
    fib 0
  fib 3
   fib 2
    fib 1
    fib 0
   fib 1


8

# Memoizzazione

In [24]:
MEM = dict()

def memoizza(f):
    def f_memoizzata(x):
        if x in MEM: return MEM[x]
        r = f(x)
        MEM[x] = r
        return r
    return f_memoizzata

@memoizza
def fib(n):
    if n == 0 or n == 1: return 1
    return fib(n-1) + fib(n-2)

fib(5), MEM

(8, {1: 1, 0: 1, 2: 2, 3: 3, 4: 5, 5: 8})